In [155]:
# !pip install selenium webdriver_manager translators undetected_chromedriver
# %pip install pymongo

In [156]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient

In [157]:
client = MongoClient('mongodb://localhost:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']

In [158]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
# driver = uc.Chrome(options=options)

In [159]:
url = 'http://localhost:3000'

In [160]:
driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [161]:
driver.get(url)
driver.maximize_window()

In [162]:
driver.find_element(By.XPATH, '/html/body/nav/a[2]').click()

In [163]:
time.sleep(5)
html = driver.page_source
soup = bs(html, 'html.parser')
div_novels = soup.find('div', class_='novelOptions')
list_novels = div_novels.find_all('a')
data_novelas = {"nombre":[], "url":[]}
for chap in list_novels:
    data_novelas['nombre'].append(chap.getText())
    data_novelas['url'].append(chap.get('href'))
df_novelas = pd.DataFrame(data_novelas)
df_novelas

,nombre,url
0,MARTIAL KING'S RETIRED LIFE,/novelas/65e7ed3d9297a4c7c5aaaeef
1,CULTIVATION CHAT GROUP,/novelas/65e874957a145405d0b24bf5
2,EMPEROR'S DOMINATION,/novelas/65efbb2ceb6e829adc4a104a
3,BECOMING IMMORTAL THROUGH GETTING MARRIED,/novelas/65efbb58eb6e829adc4a104c
4,"AS AN IMMORTAL, I ONLY LEARN FORBIDDEN SKILLS",/novelas/65f1af5d4285ef9618f6b28f
5,"BE GENTLE, IMMORTAL MASTER",/novelas/65f2715cc27445147d4bf041
6,DAO OF THE BIZARRE IMMORTAL,/novelas/65f276ae08dd1953fc6aff65
7,IT'S LONELY TO BE INVINCIBLE,/novelas/65f277ff588d402684d17d28
8,INVINCIBLE! THIS PERSON HAS LIVED FOR A BILLIO...,/novelas/65f3192d0482fb05152092b7
9,INVINCIBLE DAD OF ANOTHER WORLD,/novelas/65f31a920482fb05152092f1


In [164]:
def traducir(texto):
    contenido_p = ''
    while True:
        try:
            contenido_p = ts.translate_text(
                texto, translator='bing', to_language='es')
            break
        except Exception as e:
            print(e)
            try:
                contenido_p = ts.translate_text(
                    texto, translator='google', to_language='es')
                break
            except Exception as e:
                print(e)
                pass
            pass
    return contenido_p

In [165]:
def enviar_contenido_capitulo(novel_id,capitulo_id,texto_capitulo):
    url_contenido_capitulos = "http://localhost:8000/api/contenidocapitulos/"
    headers = {
        'Content-Type': 'application/json'
    }
    payloadchapter = json.dumps({
        "novela_id": novel_id,
        "capitulo_id": capitulo_id,
        "texto": texto_capitulo
    })
    rescap = requests.request("POST", url_contenido_capitulos, headers=headers, data=payloadchapter)
    return json.loads(str(rescap.text))['_id']

In [166]:
def manejar_driver(url, driver, novel_id):
    texto_capitulo = ''
    driver.get(url)
    list_chapter = driver.find_elements(By.CLASS_NAME, 'capitulo')
    for chapter in list_chapter:
        capitulo_id = chapter.get_attribute('id')
        print(chapter.text, capitulo_id)
        coleccion = db['app_contenidocapitulo']
        resultado = coleccion.find_one({'novela_id': novel_id, 'capitulo_id': capitulo_id})
        # print(resultado)
        # break
        if not resultado:
            chapter.click()
            time.sleep(8)
            iframe = driver.find_element(
                By.XPATH, "/html/body/div/div/div[1]/iframe")
            # Cambiar el contexto al iframe
            driver.switch_to.frame(iframe)
            html_frame = driver.page_source
            soup_frame = bs(html_frame, 'html.parser')
            div_chap_content = soup_frame.find('div', id='chr-content')
            p_chap_content = div_chap_content.find_all('p')
            chap_content = []
            for p in p_chap_content:
                chap_content.append(p.getText().strip().rstrip())
            # Volver al contexto principal de la página web
            
            for parrafo in chap_content:
                text = parrafo
                if parrafo != '':
                    text = traducir(text)
                texto_capitulo += f"<p>{text}</p>"
            _id = enviar_contenido_capitulo(novel_id=novel_id,capitulo_id=capitulo_id,texto_capitulo=texto_capitulo)
            print(_id)

            driver.switch_to.default_content()
                        
            # break
    

In [167]:
for idx, novela in df_novelas.iterrows():
    novel_id = novela.url.split('/')[-1]
    print(novel_id)
    time.sleep(4)
    manejar_driver(url=f"{url}{novela.url}", driver=driver,novel_id=novel_id)
    # break
driver.quit()

65e7ed3d9297a4c7c5aaaeef
Prologue 65efc5d005857ee1118a1612
Volume 1 1 The Imperial Court Enjoys as Many Benefits as the Turbulence Experienced in the Wulin 65efc5d305857ee1118a1613
Volume 1 2 My Shijie, Shiyi, Tai Shishu… Are All The Same Person 65efc5d705857ee1118a1614
Volume 1 3 The Door Dividing Liu Shan Men and the People at the Centre 65efc5db05857ee1118a1615
Volume 1 4 Yi Ren is at Liu Shan Men 65efc5df05857ee1118a1616
Volume 1 5 The Root of Trouble (Part 1) 65efc5e305857ee1118a1617
Volume 1 6 The Root of Trouble (Part 2) 65efc5e605857ee1118a1618
Volume 1 7 A Big Discovery 65efc5ea05857ee1118a1619
Volume 1 8 Deal at the Examination Venue 65efc5ee05857ee1118a161a
Volume 1 9 IMPORTANT 65efc5f205857ee1118a161b
Volume 1 10 My Turn 65efc5f505857ee1118a161c
Volume 1 11 Let’s Fight 65efc5f905857ee1118a161d
Volume 1 12 Liu Shan VS Qilin 65efc5fd05857ee1118a161e
Volume 1 13 Fucking Bitch VS Horseshoe Triceps 65efc60105857ee1118a161f
Volume 1 14 Goodbye Shiyi 65efc60505857ee1118a1620
Volum

AttributeError: 'NoneType' object has no attribute 'find_all'